In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
import copy
import os

np.set_printoptions(precision=3)

# for colour palette
colours = ["#C96868", "#FADFA1", "#7EACB5"]
pallete = plt.cm.colors.ListedColormap(colours)

# --- image main
save_dir = "../portfolio/themes/blog/static/images"
file_name = "2025-08-09-careful-seeding"

# we will also set the random seed to 1
seed = 1
np.random.seed(seed)

In [3]:
def phi(c, w, x, n, k):
    error = 0
    for i in range(n):
        for j in range(k):
            if w[i, j] == 1:
                error += np.sum((c[j] - x[i]) ** 2)
    return error / n